# ZBIW-Zertifikatskurs "Data Librarian"
# Modul 2.1: Strukturierte Daten und Metadaten

## Text-Dateien

### Wiederholung: Lesen und Schreiben von Textdateien in Python

Die zentrale Funktion zum Öffnen von Dateien ist `open`. Neben dem obligatorischen Dateipfad-Argument nimmt diese Funktion einen Modus-Parameter entgegen. Der Rückgabetyp der Funktion ist standardmäßig ein `TextIOWrapper`-Objekt, welches es erlaubt, den Inhalt der Datei als String (`str`) einzulesen.

Reminder: Ebenso, wie wir in der Lektion **Text-** und **Binärdatenformate** unterschieden haben, ermöglicht auch Python die Behandlung einer Datei entweder im Text- oder Binärmodus. Der Textmodus ist für die `open`-Funktion Standard (impliziter Parameter `t`) - den Binärmodus erzwingt man mit dem Parameter `b`. 

Weitere optionale Parameter erlauben die Spezifikation von Encoding oder der Behandlung von Zeilenumbrüchen. Weitere Informationen in der [Dokumentation](https://docs.python.org/3/library/functions.html#open).

In [1]:
# Datei im 'read-only' Modus öffnen (Textmodus)
file_for_reading = open('data/lorem_ipsum.txt', 'r')

In [2]:
type(file_for_reading)

_io.TextIOWrapper

In [3]:
# eine bestimmte Anzahl Zeichen einlesen mit read()
first_10_chars = file_for_reading.read(10)
first_10_chars

'Distinctio'

In [4]:
# eine ganze Zeile einlesen mit readline()
next_line = file_for_reading.readline()
next_line

' soluta deleniti quidem maiores et sed voluptas. Qui est quis libero dolor. Illo omnis quibusdam molestias et quis nam repudiandae sint.\n'

In [5]:
# den gesamten Inhalt der Datei Zeile für Zeile einlesen
whole_text = file_for_reading.readlines()
whole_text

['\n',
 'Impedit quod repellendus dolor. Earum est dolorem non voluptas aut. Cum corporis aut aspernatur consequatur vel aut sit qui. Veniam sit dolorem omnis amet facilis nobis dolor. Eum magni sunt quia libero. Fuga voluptatem et voluptatem aut veritatis possimus.\n',
 '\n',
 'Illum et deleniti nisi ex autem eum. Eveniet cum porro enim vitae sint consectetur. Fugit aut fugit inventore omnis. Quidem sint quia reprehenderit quae molestiae quia. Id facere similique et.\n',
 '\n',
 'Vero dignissimos aspernatur ex qui architecto. Ipsam repellendus sit ipsa. Nihil eligendi accusamus esse repudiandae debitis. Dolore eligendi aut enim dolorem.\n',
 '\n',
 'Esse dicta hic id. Alias alias mollitia nihil porro voluptatem sit et. Maiores sint autem numquam.\n']

In [6]:
# nach getaner Arbeit muss die Datei wieder geschlossen werden:
file_for_reading.close()

In [7]:
# besser: 'with' Block benutzen, damit Dateien automatisch geschlossen werden
with open('data/lorem_ipsum.txt', 'r') as infile:
    data = infile.readlines()

`data` ist nun ein Listenobjekt, wobei jedes Listenelement eine Textzeile aus der Datei enthält:

In [8]:
type(data)

list

In [9]:
data

['Distinctio soluta deleniti quidem maiores et sed voluptas. Qui est quis libero dolor. Illo omnis quibusdam molestias et quis nam repudiandae sint.\n',
 '\n',
 'Impedit quod repellendus dolor. Earum est dolorem non voluptas aut. Cum corporis aut aspernatur consequatur vel aut sit qui. Veniam sit dolorem omnis amet facilis nobis dolor. Eum magni sunt quia libero. Fuga voluptatem et voluptatem aut veritatis possimus.\n',
 '\n',
 'Illum et deleniti nisi ex autem eum. Eveniet cum porro enim vitae sint consectetur. Fugit aut fugit inventore omnis. Quidem sint quia reprehenderit quae molestiae quia. Id facere similique et.\n',
 '\n',
 'Vero dignissimos aspernatur ex qui architecto. Ipsam repellendus sit ipsa. Nihil eligendi accusamus esse repudiandae debitis. Dolore eligendi aut enim dolorem.\n',
 '\n',
 'Esse dicta hic id. Alias alias mollitia nihil porro voluptatem sit et. Maiores sint autem numquam.\n']

Das Iterieren über die einzelnen Zeilen ist ebenfalls möglich:

In [10]:
with open('data/lorem_ipsum.txt', 'r') as infile:
    for line in infile:
        print(line.strip()) # entferne überflüssigen Whitespace

Distinctio soluta deleniti quidem maiores et sed voluptas. Qui est quis libero dolor. Illo omnis quibusdam molestias et quis nam repudiandae sint.

Impedit quod repellendus dolor. Earum est dolorem non voluptas aut. Cum corporis aut aspernatur consequatur vel aut sit qui. Veniam sit dolorem omnis amet facilis nobis dolor. Eum magni sunt quia libero. Fuga voluptatem et voluptatem aut veritatis possimus.

Illum et deleniti nisi ex autem eum. Eveniet cum porro enim vitae sint consectetur. Fugit aut fugit inventore omnis. Quidem sint quia reprehenderit quae molestiae quia. Id facere similique et.

Vero dignissimos aspernatur ex qui architecto. Ipsam repellendus sit ipsa. Nihil eligendi accusamus esse repudiandae debitis. Dolore eligendi aut enim dolorem.

Esse dicta hic id. Alias alias mollitia nihil porro voluptatem sit et. Maiores sint autem numquam.


Man kann den Code noch etwas verkürzen, wenn man die Referenz auf `infile` nicht zusätzlich benötigt:

In [11]:
for line in open('data/lorem_ipsum.txt', 'r'):
    print(line.strip())

Distinctio soluta deleniti quidem maiores et sed voluptas. Qui est quis libero dolor. Illo omnis quibusdam molestias et quis nam repudiandae sint.

Impedit quod repellendus dolor. Earum est dolorem non voluptas aut. Cum corporis aut aspernatur consequatur vel aut sit qui. Veniam sit dolorem omnis amet facilis nobis dolor. Eum magni sunt quia libero. Fuga voluptatem et voluptatem aut veritatis possimus.

Illum et deleniti nisi ex autem eum. Eveniet cum porro enim vitae sint consectetur. Fugit aut fugit inventore omnis. Quidem sint quia reprehenderit quae molestiae quia. Id facere similique et.

Vero dignissimos aspernatur ex qui architecto. Ipsam repellendus sit ipsa. Nihil eligendi accusamus esse repudiandae debitis. Dolore eligendi aut enim dolorem.

Esse dicta hic id. Alias alias mollitia nihil porro voluptatem sit et. Maiores sint autem numquam.


Das Schreiben von Dateien funktioniert analog zum Lesen - hier wird der Modus-Parameter der `open`-Funktion auf `w` (write) gesetzt:

In [12]:
# Datei im 'write' Modus öffnen - bestehende Datei wird überschrieben!
with open('output/writing_file.txt', 'w') as outfile:
    outfile.writelines(data[0])

Alternativ sorgt der Parameterwert `a` (append) dafür, dass bestehender Inhalt in der Ausgabedatei nicht überschrieben wird. Stattdessen wird der neue Inhalt angehangen:

In [13]:
# Datei im 'append' Modus öffnen - Geschriebenes wird an bestehenden Inhalt angehangen
with open('output/appending_file.txt', 'a') as outfile:
    outfile.writelines(data[0])

---

## CSV

Wie wir gesehen haben, ist das CSV-Format ein textbasiertes Datenformat. Daher kann es wie eine reguläre Textdatei in Python behandelt werden. Dies hat allerdings seine Tücken, wie das folgende Beispiel zeigt.

### Beispiel: NYT-Beststellerliste

Als erstes sehen wir uns eine CSV-Datei ohne Header an. Hier ist der Inhalt der ersten 10 Zeilen:

In [14]:
!head data/nyt_bestsellers.csv

Wir sehen, dass jeder Datensatz aus 3 Datenfeldern besteht, die wir als Rang, Titel und Genre identifizieren. Entsprechend versuchen wir, diese Werte getrennt auszulesen:

In [15]:
# Beispiel: CSV-Datei auch als Text behandeln
for line in open('data/nyt_bestsellers.csv', 'r'):
    row = line.split(',') # Zeile am Trennzeichen splitten
    rank = row[0]  # Zuordnung der Felder
    title = row[1]
    genre = row[2].strip()
    print("Rank: {}\nTitle: {}\nGenre: {}\n".format(rank, title, genre))

Rank: "1"
Title: "I Love Dad with The Very Hungry Caterpillar"
Genre: "children"

Rank: "2"
Title: "The Wonderful Things You Will Be"
Genre: "children"

Rank: "3"
Title: "Dr. Seuss's I Love Pop!: A Celebration of Dads"
Genre: "children"

Rank: "4"
Title: "Dragons Love Tacos"
Genre: "children"

Rank: "5"
Title: "How to Babysit a Grandpa"
Genre: "children"

Rank: "6"
Title: "I Wish You More"
Genre: "children"

Rank: "7"
Title: "Grumpy Monkey"
Genre: "children"

Rank: "8"
Title: "The Day the Crayons Quit"
Genre: "children"

Rank: "9"
Title: "Dear Girl
Genre: "

Rank: "10"
Title: "Rosie Revere
Genre: Engineer (Questioneers Collection Series)"

Rank: "11"
Title: "Brawl of the Wild (Dog Man Series #6)"
Genre: "children"

Rank: "12"
Title: "The Meltdown (Diary of a Wimpy Kid Series #13)"
Genre: "children"

Rank: "13"
Title: "Harry Potter and the Sorcerer's Stone (Harry Potter Series #1)"
Genre: "children"

Rank: "14"
Title: "Escape from the Isle of the Lost (Descendants Series #4)"
Genre: "ch

### Beispiel: Bibsonomy-Export

Vorbereitung: Auf [bibsonomy](https://www.bibsonomy.org) die Ergebnisse einer Keyword-Suche (z.B. ["Bibliothek"](https://www.bibsonomy.org/export/search/Bibliothek)) als CSV exportieren (Limit ggf. vorher auf 1000 erhöhen), lokal speichern.

Diese Datei enthält einen Header, der uns die Keys zu den Werten in den Datensätzen liefert:

In [16]:
!head -n 1 data/search_Bibliothek.csv

BibliographyType,ISBN,Identifier,Author,Title,Journal,Volume,Number,Month,Pages,Year,Address,Note,URL,Booktitle,Chapter,Edition,Series,Editor,Publisher,ReportType,Howpublished,Institution,Organizations,School,Annote,Custom1,Custom2,Custom3,Custom4,Custom5


In [17]:
path = 'data/search_Bibliothek.csv'

### Naiver Ansatz über Einlesen als Text und splitten am Komma:

In [18]:
with open(path) as csvfile:
    lines = csvfile.readlines()

header = lines[0].strip().split(',') # Kopfzeile = erste Datenreihe
print(header)

['BibliographyType', 'ISBN', 'Identifier', 'Author', 'Title', 'Journal', 'Volume', 'Number', 'Month', 'Pages', 'Year', 'Address', 'Note', 'URL', 'Booktitle', 'Chapter', 'Edition', 'Series', 'Editor', 'Publisher', 'ReportType', 'Howpublished', 'Institution', 'Organizations', 'School', 'Annote', 'Custom1', 'Custom2', 'Custom3', 'Custom4', 'Custom5']


In [19]:
first_line = lines[1].strip().split(',') # 2. Datenreihe = 1. Datensatz; splitten der Datenfelder am Trennzeichen

for i in range(len(first_line)):
    print(header[i], ':', first_line[i]) # Zuordnung der Header-Felder zu den Daten

BibliographyType : 1
ISBN : ""
Identifier : "selbach_bibliothek_2007"
Author : "Selbach
Title :  Michaela & der Fachhochsch.]
Journal :  {[Bibliothek}"
Volume : "Bibliothek 2.0 : neue Perspektiven und Einsatzmöglichkeiten für wissenschaftliche Bibliotheken"
Number : ""
Month : 
Pages : 
Year : ""
Address : ""
Note : 2007
URL : "{[Köln]}"
Booktitle : ""
Chapter : ""
Edition : ""
Series : ""
Editor : ""
Publisher : ""
ReportType : ""
Howpublished : "{[Bibliothek} der Fachhochsch.]"
Institution : ""
Organizations : ""
School : ""
Annote : ""
Custom1 : ""
Custom2 : ""
Custom3 : ""
Custom4 : ""
Custom5 : "imported"


IndexError: list index out of range

Wie wir sehen, funktioniert dieser naive Ansatz nicht, da auch die Datenfelder Kommata enthalten und somit unser Trennzeichen nicht eindeutig ist. Die Aufteilung der Daten in die korrekten Felder klappt so nicht.

### Nutzen der CSV Library

Python bietet mit dem `csv` Modul eine komforable Möglichkeit, CSV Dateien zu lesen und zu schreiben.

In [20]:
import csv

In [21]:
data = []
with open(path, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=",") # csv Reader Objekt
    for row in reader:                          # ... erlaubt zeilenweises Iterieren
        data.append(row)

Randnotiz: Die drei Zeilen

```
reader = csv.reader(csvfile, delimiter=",")
    for row in reader:
        data.append(row)
```

können in Kurzform mit einer sogenannten *list comprehension* auch so ausgedrückt werden:

```
data = [row for row in csv.reader(csvfile, delimiter=",")]
```

In [22]:
header = data[0] # Kopfzeile ist auch hier wieder die erste Datenreihe
print(header)

['BibliographyType', 'ISBN', 'Identifier', 'Author', 'Title', 'Journal', 'Volume', 'Number', 'Month', 'Pages', 'Year', 'Address', 'Note', 'URL', 'Booktitle', 'Chapter', 'Edition', 'Series', 'Editor', 'Publisher', 'ReportType', 'Howpublished', 'Institution', 'Organizations', 'School', 'Annote', 'Custom1', 'Custom2', 'Custom3', 'Custom4', 'Custom5']


In [23]:
first_line = data[1] # zweite Datenreihe = 1. Datensatz
for i in range(len(first_line)):
    print(header[i], ':', first_line[i])

BibliographyType : 1
ISBN : 
Identifier : selbach_bibliothek_2007
Author : Selbach, Michaela & der Fachhochsch.], {[Bibliothek}
Title : Bibliothek 2.0 : neue Perspektiven und Einsatzmöglichkeiten für wissenschaftliche Bibliotheken
Journal : 
Volume : 
Number : 
Month : 
Pages : 
Year : 2007
Address : {[Köln]}
Note : 
URL : 
Booktitle : 
Chapter : 
Edition : 
Series : 
Editor : 
Publisher : {[Bibliothek} der Fachhochsch.]
ReportType : 
Howpublished : 
Institution : 
Organizations : 
School : 
Annote : 
Custom1 : 
Custom2 : 
Custom3 : imported
Custom4 : 
Custom5 : 


Nun hat die Zuordnung der Daten zu den Header Feldern korrekt geklappt.

Eine weitere praktische Funktion des CSV-Moduls ist die Möglichkeit, die Daten unmittelbar in ein `Dictionary` einzulesen. Dies setzt voraus, dass die Datei einen Header enthält, welcher die Keys für das Dictionary bereitstellt.

In [ ]:
# einlesen mit DictReader
with open(path, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        print("Title: {}\nAuthors: {}\nISBN: {}\n".format(row['Title'], row['Author'], row['ISBN']))

Im Falle des fehlenden Headers werden die Keys manuell definiert, und dem Reader im `fieldnames`-Parameter mitgeteilt:

In [ ]:
header=['rank', 'title', 'genre']

with open('data/nyt_bestsellers.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', fieldnames=header)
    for row in reader:
        print("Rank: {}\nTitle: {}\nGenre: {}\n".format(row['rank'], row['title'], row['genre']))

---

## JSON

Wir sparen uns jetzt den Versuch, JSON Dateien ebenfalls wie normale Textdateien zu behandeln, da die hierarchische Strukur ungleich schwerer manuell zu parsen wäre.

Stattdessen nutzen wir auch hier das passende Python Modul.

In [26]:
import json

Als Datensatz verwenden wir wieder [bibsonomy](https://www.bibsonomy.org/export/search/Bibliothek). Wir stellen das Exportformat auf JSON um und stellen fest, dass uns diesmal kein Download angeboten wird, sondern wir auf eine neue URL umgeleitet werden. Diese notieren wir:

In [27]:
url = "https://www.bibsonomy.org/json/search/Bibliothek?items=1000&duplicates=merged"

Es gibt nun zwei mögliche Wege, diese Daten über Python von der url abzurufen.

Mit der built-in library urllib:

In [28]:
import urllib.request
import urllib.parse

f = urllib.request.urlopen(url) # f ist eine HTTPResponse
result = f.read().decode('utf-8') # utf-8 ist oft die richtige Wahl

In [29]:
print(type(result))

<class 'str'>


In [30]:
# Ergebnis ist str, muss erst noch als JSON verarbeitet werden (Ergebnis = dict)
data = json.loads(result) # json.loads() lädt JSON Daten aus einem String
print(type(data))

<class 'dict'>


Mit third-party library requests:

In [31]:
import requests

In [32]:
result = requests.get(url) # result ist ein requests.models.Response Objekt

In [33]:
data = result.json() # das Response-Objekt bietet diese nützliche Methode an

In [34]:
type(data) # Ergebnis = dict

dict

In [ ]:
print(data)

In [ ]:
# besser lesbare Ausgabe mittels Einrückungen
print(json.dumps(data, indent=2))

In [ ]:
# das gleiche, aber durch Nutzung des pprint (pretty print) Moduls
from pprint import pprint
pprint(data)

Wir können die Daten nun wie ein normales Dictionary behandeln und damit arbeiten:

In [38]:
print(data.keys()) # Anzeige der vorhandenen Keys

dict_keys(['types', 'properties', 'items'])


In [39]:
items = data['items']
print(type(items))
print(len(items))

<class 'list'>
1914


In [40]:
publications = [item for item in items if item['type'] == 'Publication']
print(len(publications))

914


In [41]:
pubs_with_abstracts = [item for item in publications if 'abstract' in item]
print(len(pubs_with_abstracts))

139


Wir wollen nun die Publikationen nach Sprache filtern und nur die Deutschen behalten. Zum Detektieren der Sprache benötigen wir ein weiteres Modul:

In [42]:
from langdetect import detect

Das `langdetect` Modul bietet eine einfache, wenn auch nicht immer hundertprozentig korrekte, Methode, textuelle Daten auf ihre Sprache zu überprüfen. Die zentrale Funktion ist dabei `langdetect.detect`, die einen String übergeben bekommt, und einen ISO 639-1 Code zurückgibt, der die Sprachen angibt, also etwa 'de', 'en' oder 'fr'.

In [43]:
ex1 = "Ein Beispielstring auf Deutsch"
detect(ex1)

'de'

In [44]:
ex2 = "Some sample text in English"
detect(ex2)

'en'

Mithilfe dieser Funktion kann man nun beispielweise textuelle Daten filtern und nur diejenigen weiterverarbeiten, die in einer bestimmten Sprache verfasst sind. In diesem Fall wollen wir nur Publikationen behalten, deren Titel auf Deutsch ist.

In [45]:
german_pubs = [item for item in publications if detect(item['label']) == 'de']

In [46]:
len(german_pubs)

780

### JSON-Dateien schreiben

JSON-Objekte können zur Persistierung natürlich auch wieder als Textdatei gespeichert werden. Hierfür steht die Funktion `json.dump` zur Verfügung, welche als Parameter die Daten (Dictionary oder List), ein Datei(-ähnliches) Objekt, sowie optional die Anzahl der Leerzeichen für Einrückung annimmt:

In [47]:
with open('output/german_publications.json', 'w') as outfile:
    json.dump(german_pubs, outfile, indent=4)

Parallel zu `json.dump` gibt es auch noch die Funktion `json.dumps`, die wir oben schon gesehen haben. Diese gibt den JSON-String zurück wie er in eine Datei geschrieben werden würde, ohne dies tatsächlich zu tun. So kann man ggf. das Ausgabe-Ergebnis vorher überprüfen:

In [ ]:
print(json.dumps(german_pubs, indent=4))

## Hausaufgabe

Suchen Sie auf [bibsonomy](https://www.bibsonomy.org) mit einem selbst gewählten Suchbegriff nach Publikationen. Wählen Sie einen deutschen Suchbegriff, um möglichst viele deutschsprachige Ergebnisse zu erhalten.

Greifen Sie mittels der JSON-API auf die Daten zu. Verarbeiten Sie die Publikationsdaten so, dass Sie ausschließlich deutschsprachige Publikationen beibehalten, welche Sie dann in verschiedene Formate überführen.

### Lingo-Exportformat erstellen

Das Eingabeformat für Lingo sieht folgendermaßen aus:

```
[id.]
text.

[id.]
text.

<...>
```

Die ID muss dabei numerisch und eindeutig sein.

Für die Verwendung von Text bieten sich Titel und, wenn vorhanden, Abstract der Publikationen an.

Speichern Sie das Ergebnis in einer Datei `bibsonomy_to_lingo.txt`.

### Solr-Exportformat erstellen

Solr kann dankenswerterweise JSON-Daten direkt als Import verwenden. Allerdings bietet es sich hier an, die bibsonomy-Daten zunächst noch etwas zu bereinigen. Überlegen Sie, welche Felder Sie möglicherweise umbenennen wollen, und welche Sie für Solr nicht benötigen. Erstellen Sie einen sauberen Export im JSON-Format.

Hier ein Beispiel, das zeigt, wie die grundsätzliche Struktur aussehen soll:

```json
[
  {
    "id" : "978-0641723445",
    "cat" : ["book","hardcover"],
    "name" : "The Lightning Thief",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 1,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 12.50,
    "pages_i" : 384
  }
,
  {
    "id" : "978-1423103349",
    "cat" : ["book","paperback"],
    "name" : "The Sea of Monsters",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 2,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 6.49,
    "pages_i" : 304
  }
]
```

## Tipps und Hinweise

In [57]:
# mit enumerate kann man über eine Liste iterieren und erhält gleichzeitig einen Zähler:
some_list = ['this', 'is', 'a', 'list']
for index, item in enumerate(some_list):
    print(index, item)

0 this
1 is
2 a
3 list


In [61]:
# Keys in Dictionaries lassen sich umbenennen...
some_dict = {'name':'Tobias', 'age': 31}
print(some_dict)
some_dict['firstname'] = some_dict.pop('name')
print(some_dict)

{'name': 'Tobias', 'age': 31}
{'age': 31, 'firstname': 'Tobias'}


In [62]:
# ... und entfernen:
some_dict.pop('age', None)
print(some_dict)

{'firstname': 'Tobias'}
